# Generative Adversarial Networks

## MNIST-fashion GAN
The Fashion-MNIST dataset contains 60,000 training images (and 10,000 test images) of fashion and clothing items, taken from 10 classes. Each image is a standardized 28×28 size in grayscale (784 total pixels).

<center>
<img src="https://miro.medium.com/max/1400/0*ga9XgppZI_XULBRz">
</center>

## Import Libraries

In [11]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, InputLayer, Conv2D, Dropout, Convolution2DTranspose, Reshape, Flatten
from tensorflow.keras.models import Sequential 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow

Check the GPU of your machine (Tesla P100 is the highest you can get)

In [ ]:
!nvidia-smi

In [13]:
print("Num GPUs Available ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available  1


## Load the Fashion MNIST Dataset

In [14]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

## Data Preprocessing

Merge the training and testing sets, Then normalize the images from [0,255] to [0,1]

In [ ]:
dataset = np.concatenate([x_train, x_test], axis = 0)
print(dataset.shape)
dataset = np.expand_dims(dataset, -1).astype("float32") / 255

Now we will reshape the data to dimensions needed by the CNN layers that's why we convert it to TensorFlow type. After that we will shuffle and batch it.

In [16]:
BATCH_SIZE = 64
dataset = tf.data.Dataset.from_tensor_slices(dataset)
dataset = dataset.shuffle(buffer_size=1024).batch(BATCH_SIZE)

## Building The Generator
The generator's input is a noise vector. 

In [ ]:
NOISE_DIM = 150

generator = Sequential ([
      InputLayer(input_shape=(NOISE_DIM,)),
      Dense(7*7*256),
      Reshape(target_shape=(7, 7, 256)),
      Convolution2DTranspose(256, 3, activation="LeakyReLU", strides=2, padding="same"),
      Convolution2DTranspose(128, 3, activation="LeakyReLU", strides=2, padding="same"),
      Convolution2DTranspose(1, 3, activation="sigmoid", padding="same")
])

generator.summary()

## Building The Discriminator

In [ ]:
discriminator = Sequential ([
      InputLayer(input_shape=((28, 28, 1))),
      Conv2D(256, 3, activation="relu", strides=2, padding="same"),
      Conv2D(128, 3, activation="relu", strides=2, padding="same"),
      Flatten(),
      Dense(64, activation="relu"),
      Dropout(0.2),
      Dense(1, activation="sigmoid")
])

discriminator.summary()

## Optimizers

In [19]:
optimizer_g = Adam(learning_rate=0.00001, beta_1=0.5)
optimizer_d = Adam(learning_rate=0.00003, beta_1=0.5)

loss_func = BinaryCrossentropy(from_logits=True)

g_acc_metric = BinaryAccuracy()
d_acc_metric = BinaryAccuracy()

## Set Up Discriminator Training

In [20]:
def train_d_step(data):
  """ train func"""
  batch_size = tf.shape(data)[0]

  noise =  tf.random.normal(shape=(batch_size, NOISE_DIM))

  y_true = tf.concat(
      [
       tf.ones(batch_size, 1),
       tf.zeros(batch_size, 1)
      ], axis = 0
  )

  with tf.GradientTape() as tape:
    fake = generator(noise)
    x = tf.concat([data, fake], axis=0)
    y_pred = discriminator(x) # D(x)
    discriminator_loss = loss_func(y_true,  y_pred)

  grads = tape.gradient(discriminator_loss, discriminator.trainable_weights)
  optimizer_d.apply_gradients(zip(grads, discriminator.trainable_weights))

  d_acc_metric.update_state(y_true, y_pred)

  visualization_loss = {
      'discriminator_loss' : discriminator_loss,
      'discriminator_acc' : d_acc_metric.result()
  }

  return visualization_loss


## Set Up Generator Training

In [21]:
def train_g_step(data):
  """ train func"""
  batch_size = tf.shape(data)[0]

  noise =  tf.random.normal(shape=(batch_size, NOISE_DIM))

  y_true = tf.ones(batch_size, 1)

  with tf.GradientTape() as tape:
    y_pred = discriminator(generator(noise)) # D(G(Z))
    generator_loss = loss_func(y_true,  y_pred)

  grads = tape.gradient(generator_loss, generator.trainable_weights)
  optimizer_g.apply_gradients(zip(grads, generator.trainable_weights))

  g_acc_metric.update_state(y_true, y_pred)
  visualization_loss = {
      'generator_loss' : generator_loss,
      'generator_acc' : g_acc_metric.result()
  }
  
  return visualization_loss

## Visualization

In [22]:
def plot_images(model):
  """ visualize generated images as we train"""
  images = model(np.random.normal(size=(81, NOISE_DIM)))

  plt.figure(figsize=(9, 9))

  for i, image in enumerate(images):
    plt.subplot(9, 9, i+1)
    plt.imshow(np.squeeze(image, -1), cmap="Greys_r")
    plt.axis('off')

  plt.show()

## Training

In [ ]:
for epoch in range(30):

  d_loss_sum = 0
  d_acc_sum = 0
  
  g_loss_sum = 0
  g_acc_sum = 0
  
  counter = 0

  for batch in dataset:
    d_loss = train_d_step(batch)

    d_loss_sum += d_loss['discriminator_loss']
    d_acc_sum += d_loss['discriminator_acc']

    g_loss = train_g_step(batch)

    g_loss_sum += g_loss['generator_loss']
    g_acc_sum += g_loss['generator_acc']

    counter += 1

  print("Epoch:{} Loss G:{:0.4f}, Loss D:{:0.4f}, Acc G:%{:0.2f}, Acc D:%{:0.2f}".format(
      epoch,
      g_loss_sum / counter,
      d_loss_sum / counter,
      100 * (g_acc_sum / counter),
      100 * (d_acc_sum / counter),
  ))
  if epoch % 2 == 0:
    plot_images(generator)

## Test The Generator

In [ ]:
images = generator(np.random.normal(size=(81, NOISE_DIM)))

plt.figure(figsize=(9, 9))

for i, image in enumerate(images):
  plt.subplot(9, 9, i+1)
  plt.imshow(np.squeeze(image, -1), cmap="Greys_r")
  plt.axis('off')

plt.show()